In [1]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/labelled_data.csv')
pd.set_option('display.max_columns', None)
# df.head()

In [ ]:
# print(df.info())
# print()
# print(df.shape)
# print()
# df.describe()
# print()
# for i in df.columns:
#     print(i,'---->',df[i].nunique())

In [3]:
from pycaret.classification import *
from imblearn.over_sampling import RandomOverSampler


ignore = ['Unnamed: 0','Anomaly_Score','timestamps']

# Initialize the setup
clf1 = setup(data=df,
             target='Anomaly',
             session_id=123,
             train_size=0.7,
             preprocess=True,
             imputation_type='simple',
             categorical_imputation='mode',
             numeric_imputation='mean',
             normalize=True,
             ignore_features=ignore,
             normalize_method='robust',
             combine_rare_levels=True,
             rare_level_threshold=0.1,
             data_split_stratify=True,
             fold_strategy='stratifiedkfold',
             fold=3,
             fold_shuffle=True,
             n_jobs=None,
             silent=True,
             fix_imbalance=True,
             fix_imbalance_method=RandomOverSampler(),
             ignore_low_variance=True,
             handle_unknown_categorical=True,
             unknown_categorical_method='least_frequent',
             feature_selection=True,
             feature_selection_method= "classic",
             log_experiment=True,
             experiment_name='anomalyDetection001'
             )

,Description,Value
0,session_id,123
1,Target,Anomaly
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(17280, 34)"
5,Missing Values,True
6,Numeric Features,20
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


Traceback (most recent call last):
  File "c:\Users\2401-00010\.conda\envs\pycaretTest\lib\site-packages\mlflow\store\tracking\file_store.py", line 271, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "c:\Users\2401-00010\.conda\envs\pycaretTest\lib\site-packages\mlflow\store\tracking\file_store.py", line 405, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\2401-00010\.conda\envs\pycaretTest\lib\site-packages\mlflow\store\tracking\file_store.py", line 1109, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\Users\2401-00010\.conda\envs\pycaretTest\lib\site-packages\mlflow\store\tracking\file_store.py", line 1102, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\Users\2401-00010\.conda\envs\pycaretTest\lib\site-packages\mlflow\utils\file_utils.py", line 183, in read_yaml
    raise MissingConfigException("Yaml file '%s' does n

In [ ]:
X_train = get_config('X_train')
X_test = get_config('X_test')

X_train.to_csv("D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/X_train.csv")
X_test.to_csv("D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/X_test.csv")

In [ ]:
# models()

In [ ]:
# Compare models to find the best one
top3 = compare_models(n_select=3)

In [ ]:
top3_tuned = [tune_model(i) for i in top3]

In [ ]:
# models(type='ensemble').index.tolist()
ensembled_models = compare_models(include= models(type='ensemble').index.tolist(), fold = 3)

In [ ]:
gbc = create_model('gbc', fold=3)
tuned_gbc = tune_model(gbc)

In [ ]:
# Analyse Model

plot_model(tuned_gbc)

In [ ]:
plot_model(tuned_gbc,plot = 'confusion_matrix')

In [ ]:
plot_model(tuned_gbc, plot = 'boundary')

In [ ]:
evaluate_model(tuned_gbc)

In [ ]:
pred_holdout = predict_model(tuned_ada)

In [ ]:
from pycaret.classification import get_leaderboard
leaderboard = get_leaderboard()

In [ ]:
# Tune the Adaboost Classifier model
# import warnings
# warnings.filterwarnings('ignore')
tuned_ada_model = tune_model(best_model,n_iter=3,optimize='Accuracy',verbose=True)

In [ ]:
# Access transformed training data
transformed_train_data = get_config('X_train')
transformed_test_data = get_config('X_test')

In [ ]:
train_predictions = tuned_ada.predict(transformed_train_data)
test_predictions = tuned_ada.predict(transformed_test_data)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(round(accuracy_score(get_config('y_train'),train_predictions),4))
print(round(accuracy_score(get_config('y_test'),test_predictions),4))


In [ ]:
predict_model(tuned_ada)

In [ ]:
import mlflow

mlflow.set_experiment(experiment_name='training_experiment_1')
mlflow.end_run()

with mlflow.start_run():
    clf1 = setup(data=df,target='Anomaly',session_id=123,train_size=0.7,
             preprocess=True,
             imputation_type='simple',
             categorical_imputation='mode',
             numeric_imputation='mean',
             normalize=True,
             ignore_features=ignore,
             normalize_method='robust',
             combine_rare_levels=True,
             rare_level_threshold=0.1,
             data_split_stratify=True,
             fold_strategy='stratifiedkfold',
             fold=3,
             fold_shuffle=True,
             n_jobs=None,
             silent=True,fix_imbalance=True,
             fix_imbalance_method=RandomOverSampler(),
             ignore_low_variance=True,
             handle_unknown_categorical=True,
             unknown_categorical_method='least_frequent',
             feature_selection=True,
             feature_selection_method= "classic",
             )
    best = compare_models(include=['lightgbm','ada','rf'])

In [ ]:
! mlflow ui